# 1. Predicting Heart Disease Using a Classification Tree

## 1.1 Data Cleaning

In [ ]:
library(tidyverse)   # Incluye dplyr, ggplot2, readr
library(readxl)      # Leer Excel 




In [56]:
data <- read_csv("C:/Users/VICTOR/Documents/GitHub/Decision_Trees/R/input/processed.cleveland.data", col_names = FALSE)

Rows: 303 Columns: 14
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): X12, X13
dbl (12): X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X14

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [37]:
head(data)  

age,sex,cp,restbp,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,hd
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0
41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0
56,1,2,120,236,0,0,178,0,0.8,1,0.0,3.0,0


In [24]:
names(data)

[1] "X1"  "X2"  "X3"  "X4"  "X5"  "X6"  "X7"  "X8"  "X9"  "X10" "X11" "X12"
[13] "X13" "X14"

In [57]:
nombres_nuevos <- c('age', 'sex', 'cp', 'restbp', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'hd')
colnames(data) <- nombres_nuevos
names(data)

[1] "age"     "sex"     "cp"      "restbp"  "chol"    "fbs"     "restecg"
 [8] "thalach" "exang"   "oldpeak" "slope"   "ca"      "thal"    "hd"

In [58]:
data <- data %>%
  mutate(across(where(is.character), ~ na_if(., "?")))
data <- data %>%
    drop_na()

In [59]:
str(data)

tibble [297 × 14] (S3: tbl_df/tbl/data.frame)
 $ age    : num [1:297] 63 67 67 37 41 56 62 57 63 53 ...
 $ sex    : num [1:297] 1 1 1 1 0 1 0 0 1 1 ...
 $ cp     : num [1:297] 1 4 4 3 2 2 4 4 4 4 ...
 $ restbp : num [1:297] 145 160 120 130 130 120 140 120 130 140 ...
 $ chol   : num [1:297] 233 286 229 250 204 236 268 354 254 203 ...
 $ fbs    : num [1:297] 1 0 0 0 0 0 0 0 0 1 ...
 $ restecg: num [1:297] 2 2 2 0 2 0 2 0 2 2 ...
 $ thalach: num [1:297] 150 108 129 187 172 178 160 163 147 155 ...
 $ exang  : num [1:297] 0 1 1 0 0 0 0 1 0 1 ...
 $ oldpeak: num [1:297] 2.3 1.5 2.6 3.5 1.4 0.8 3.6 0.6 1.4 3.1 ...
 $ slope  : num [1:297] 3 2 2 3 1 1 3 1 2 3 ...
 $ ca     : chr [1:297] "0.0" "3.0" "2.0" "0.0" ...
 $ thal   : chr [1:297] "6.0" "3.0" "7.0" "3.0" ...
 $ hd     : num [1:297] 0 2 1 0 0 0 3 0 2 1 ...


### Categoricas a transformar: 
    -cp
    -restecg
    -slope
    -ca
    -thal
    -hd

In [47]:
variables_categoricas <- c("cp","restecg","slope","ca","thal","hd")
data[variables_categoricas] <- lapply(data[variables_categoricas], as.factor)